In [1]:
# import pymysql
# from sqlalchemy import create_engine
# import sys
# from abc import ABC, abstractmethod
# from typing import *

import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 0. 모듈의 상위 패키지를 파이썬 경로로 설정

In [2]:
import sys

In [3]:
sys.path

['/Users/minjoo/codeStudy/psudonymizerTeam/pseudonymizer/tests',
 '/opt/homebrew/anaconda3/lib/python311.zip',
 '/opt/homebrew/anaconda3/lib/python3.11',
 '/opt/homebrew/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/anaconda3/lib/python3.11/site-packages',
 '/opt/homebrew/anaconda3/lib/python3.11/site-packages/aeosa']

In [4]:
# sys.path.append("C:\Pseudonymizer\pseudonymizer")
sys.path.append("/Users/minjoo/codeStudy/psudonymizerTeam/pseudonymizer")
# /path/to/your/module

#### 1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드

In [5]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [8]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "1234")

In [9]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")

In [11]:
SQL = input("SQL 쿼리문 입력변수 = ")
    # SELECT * FROM DATA_FINANCE

In [12]:
queryObject.dataQueryLanguage(sql = SQL)

In [ ]:
# DATA_FINANCE_TUPLE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [13]:
DATA_FINANCE_TABLE = queryObject.executeQueryAsDataFrame()

In [14]:
type(DATA_FINANCE_TABLE)
DATA_FINANCE_TABLE.info()
DATA_FINANCE_TABLE.tail(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NUM_SERIAL                  95000 non-null  int64 
 1   NAME                        95000 non-null  object
 2   GENDER                      95000 non-null  object
 3   AGE                         95000 non-null  int64 
 4   PHONE_NUMBER                95000 non-null  object
 5   ZIP_CODE                    95000 non-null  int64 
 6   HOME_ADDRESS                95000 non-null  object
 7   HOME_TYPE                   95000 non-null  object
 8   INCOME_BRACKET              95000 non-null  int64 
 9   CREDIT_SCORE                95000 non-null  int64 
 10  REPAYMENT_RISK_INDEX        95000 non-null  int64 
 11  AMT_CREDITCARD_PAYMENT      95000 non-null  int64 
 12  AMT_CASHADVANCE_PAYMENT     95000 non-null  int64 
 13  NUM_CREDITCARD_ISSUANCES    95000 non-null  in

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,...,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION,SERIALNUM
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,...,8488084,7377056,8,2,N,0,0,N,0,F94998
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,...,5109440,3625570,3,2,N,0,0,N,0,F94999
94999,94999,안전웅,male,46,010-9622-1090,10585,경기도 고양시 덕양구 안진6길 12(지축동),오피스텔,7,44,...,874939,5785351,7,0,N,0,0,N,0,F95000


#### 2. 가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환

In [15]:
from pseudonymizer.pseudonymizer import Pseudonymizer

In [22]:
from pseudonymizer.pseudonym import Pseudonym, PseudonymizerModule
    # ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)

In [17]:
help(Pseudonym)

Help on class Pseudonym in module pseudonymizer.pseudonym:

class Pseudonym(builtins.object)
 |  Pseudonym(dataframe, pseudonymizer_module)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe, pseudonymizer_module)
 |      원본정보(재현데이터)와 가명처리 구체 클래스를 인스턴스 변수로 선언하는(초기화) 생성자
 |  
 |  __str__(self)
 |      캡슐화된 데이터셋의 속성(컬럼)정보를 반환하는 메서드
 |  
 |  addDictionary(self, column, pseudonymizers)
 |      가명처리를 수행할 데이터 컬럼명과 해당 열에 적용할 여러 가명처리 기법 리스트를 입력받아 다양한 비식별 조치를 수행할 수 있도록 지정하는 메서드
 |  
 |  addPseudonymizer(self, pseudonymizer)
 |      가명처리 추상 클래스에 대한 자식 클래스를 입력받는 pseudonymizer파라미터를 가지는 메서드
 |  
 |  categorizeEquivalentClass(self, attributes: List[str])
 |      각 행(레코드)에 대한 개인식별가능정보 속성(컬럼)들 사이에 동질 집합을 확인하는 메서드
 |      Pseudonym(dataframe).equivalent_class.keys()를 통해 동질집합 확인
 |  
 |  countEquivalentClass(self)
 |  
 |  getPseudonymizedDataframe(self)
 |      가명처리 데이터 반환
 |  
 |  pseudonymizeData(self)
 |      가명처리 기법을 해당 컬럼에 적용하는 메서드(apply함수를 활용하여 데이터프레임 모든 행, 특정 열에 비식별조치를 취하는 접근방식)
 |

In [129]:
from pseudonymizer.pseudonymizers.maskingPseudonymizer import MaskingPseudonymizer
from pseudonymizer.pseudonymizers.charcategorization import CategorizationOfCharacter
from pseudonymizer.pseudonymizers.numcategorization import CategorizationOfNumeric
from pseudonymizer.pseudonymizers.columncategorization import CategorizationOfColumn
from pseudonymizer.pseudonymizers.microAggregation import MicroAggregation
from pseudonymizer.pseudonymizers.randomRoundingPseudonymizer import RandomRoundingPseudonymizer
from pseudonymizer.pseudonymizers.topandBottomCoding import TopandBottomCoding

In [25]:
pseudonymizer_mod = PseudonymizerModule()

NameError: name 'data_type' is not defined

In [130]:
# 가명처리 기법 적용 객체 생성
pseudonym_object = Pseudonym(
    dataframe = DATA_FINANCE_TABLE,
    pseudonymizer_module = None
)

In [16]:
# 원본 데이터의 컬럼 정보 출력(.info())
# print(pseudonym_object)

In [131]:
# 속성값 조합을 이용한 동질집합 생성
pseudonym_object.categorizeEquivalentClass(
    attributes = ["NUM_CREDITCARD_ISSUANCES", "TF_LOAN", "TF_PENSION"]
)

In [132]:
# 동질집합 반환
pseudonym_object.countEquivalentClass()

(1, 'N', 'N') 7951
(1, 'N', 'Y') 596
(1, 'Y', 'N') 840
(1, 'Y', 'Y') 83
(2, 'N', 'N') 7988
(2, 'N', 'Y') 580
(2, 'Y', 'N') 883
(2, 'Y', 'Y') 54
(3, 'N', 'N') 8031
(3, 'N', 'Y') 585
(3, 'Y', 'N') 895
(3, 'Y', 'Y') 69
(4, 'N', 'N') 7960
(4, 'N', 'Y') 627
(4, 'Y', 'N') 902
(4, 'Y', 'Y') 68
(5, 'N', 'N') 7972
(5, 'N', 'Y') 573
(5, 'Y', 'N') 923
(5, 'Y', 'Y') 82
(6, 'N', 'N') 7857
(6, 'N', 'Y') 618
(6, 'Y', 'N') 868
(6, 'Y', 'Y') 71
(7, 'N', 'N') 7887
(7, 'N', 'Y') 593
(7, 'Y', 'N') 878
(7, 'Y', 'Y') 77
(8, 'N', 'N') 8002
(8, 'N', 'Y') 611
(8, 'Y', 'N') 803
(8, 'Y', 'Y') 64
(9, 'N', 'N') 7967
(9, 'N', 'Y') 598
(9, 'Y', 'N') 890
(9, 'Y', 'Y') 50
(10, 'N', 'N') 7899
(10, 'N', 'Y') 639
(10, 'Y', 'N') 906
(10, 'Y', 'Y') 60


In [133]:
categorize_col = CategorizationOfColumn(numeric_type="bin", grouping_standard=["A", "B", "C"], right=True, ascending=False)

In [134]:
micro_aggregation = MicroAggregation(calulate_type="average", quasi_identifier=(1, 'N', 'N'))

In [125]:
# 가명처리 기법 클래스 입력
pseudonym_object.addPseudonymizer(
    pseudonymizer = categorize_col
)

In [110]:
# 적용 컬럼과 여러 가명처리 기법 정의
pseudonym_object.addDictionary(
    # column = "AGE",
    column = "ZIP_CODE",
    pseudonymizers = [categorize_col]
)

In [135]:
pseudonym_object.addDictionary(
    column = "CREDIT_SCORE",
    pseudonymizers = [micro_aggregation]
)

In [83]:
pseudonym_object._pseudonymizers

In [136]:
pseudonym_object._pseudonymDictionary

{'CREDIT_SCORE': [<pseudonymizer.pseudonymizers.microAggregation.MicroAggregation at 0x17abbddd0>]}

In [103]:
pseudonym_object.equivalent_class

{(1, 'N', 'N'): [13,
  27,
  31,
  36,
  50,
  51,
  61,
  63,
  70,
  76,
  85,
  95,
  112,
  121,
  139,
  150,
  162,
  164,
  170,
  173,
  181,
  186,
  187,
  189,
  192,
  194,
  196,
  216,
  223,
  246,
  256,
  259,
  268,
  269,
  274,
  281,
  293,
  299,
  326,
  331,
  337,
  364,
  375,
  399,
  401,
  412,
  417,
  420,
  426,
  430,
  431,
  435,
  436,
  449,
  460,
  469,
  470,
  477,
  511,
  515,
  525,
  529,
  539,
  578,
  586,
  597,
  624,
  635,
  636,
  641,
  659,
  678,
  688,
  692,
  693,
  702,
  711,
  718,
  721,
  727,
  752,
  756,
  760,
  765,
  799,
  821,
  842,
  844,
  846,
  860,
  869,
  873,
  883,
  896,
  904,
  906,
  911,
  936,
  938,
  961,
  974,
  975,
  998,
  1015,
  1032,
  1037,
  1039,
  1054,
  1055,
  1067,
  1076,
  1093,
  1098,
  1100,
  1110,
  1113,
  1119,
  1124,
  1149,
  1180,
  1192,
  1201,
  1203,
  1209,
  1218,
  1223,
  1225,
  1236,
  1248,
  1255,
  1268,
  1280,
  1288,
  1291,
  1311,
  1331,
  1356,
  13

In [137]:
# 가명처리 수행
pseudonym_object.pseudonymizeData()

In [138]:
# 가명정보 반환
pseudonym_data = pseudonym_object.getPseudonymizedDataframe()

In [139]:
pseudonym_data

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,...,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION,SERIALNUM
0,0,강현심,female,C,010-1826-4535,B,충청북도 제천시 수산면 옥순봉로6길 61-32,"판잣집, 비닐하우스",6,441.000000000000000000000000000000,...,5357418,8583708,3,5,N,0,0,N,0,F1
1,1,서삼봉,female,A,010-1765-1467,A,경기도 광주시 초월읍 도곡길 109-9,다세대주택,8,515.000000000000000000000000000000,...,8870904,5328233,3,1,N,0,0,N,0,F2
2,2,팽상아,female,B,010-6593-5091,C,경상남도 창원시 마산회원구 회원남32길 28-1(회원동),"판잣집, 비닐하우스",1,269.000000000000000000000000000000,...,8525658,2005663,6,5,N,0,0,N,0,F3
3,3,박옥점,female,A,010-7714-1345,C,전라남도 무안군 청계면 구로길 65,다가구 단독주택,7,254.000000000000000000000000000000,...,2593007,8944424,4,1,N,0,0,N,0,F4
4,4,윤경념,female,C,010-5360-6014,B,충청남도 당진시 신평면 신평길 64-6,영업 겸용 단독주택,8,590.000000000000000000000000000000,...,411525,5125357,5,3,N,0,0,N,0,F5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,94995,갈재희,female,C,010-9796-1142,B,경상북도 영덕군 달산면 청련길 10-10,다세대주택,8,868.000000000000000000000000000000,...,835585,6144642,5,1,N,0,0,Y,0,F94996
94996,94996,림상영,male,B,010-2817-1504,B,강원특별자치도 영월군 무릉도원면 황정길 253-11,기타,7,875.000000000000000000000000000000,...,4691445,6000005,5,3,N,0,0,N,0,F94997
94997,94997,명시라,female,C,010-2536-2511,A,경기도 부천시 장말로 292(심곡동),연립주택,4,294.000000000000000000000000000000,...,8488084,7377056,8,2,N,0,0,N,0,F94998
94998,94998,롤미라,female,C,010-5091-5053,B,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986.000000000000000000000000000000,...,5109440,3625570,3,2,N,0,0,N,0,F94999


#### 3. 가명처리 절차 : 가명정보에 프라이버시 보호 모델 적용

In [25]:
from pseudonymizer.privacyPreservingModels import PrivacyPreservingModel

In [27]:
help(PrivacyPreservingModel)

Help on class PrivacyPreservingModel in module pseudonymizer.privacyPreservingModels:

class PrivacyPreservingModel(builtins.object)
 |  PrivacyPreservingModel(dataframe, epsilon)
 |  
 |  개인식별가능정보 속성을 기준으로 그룹화된 데이터로 프라이버시 보호 모델을 적용하여 정량적인 위험성을 규정하는 실행 클래스
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataframe, epsilon)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      동질집합에 대한 정보를 문자열로 반환하는 메서드
 |  
 |  applyDPrivacy(self, method, boundary, attributes, sensitive_attribute, outlier)
 |      차분 프라이버시 기법(차등적 정보보호 기능)을 적용하는 메서드
 |  
 |  applyKAnonymityOrLDiversity(self, method: str, K: int, L: int, attributes: List[str], sensitive_attribute)
 |      K-익명성과 L-다양성 모델을 선택적으로 적용하는 메서드
 |      input
 |      -----
 |      method: 프라이버시 보호 모델 메서드를 받고, 
 |      keyword arguments에 딕셔너리 형식으로 각 기법에 필요한 파라미터를 받아옴
 |  
 |  applyLocalLDiversity(self, K: int, L: int, attributes: List[str], sensitive_attribute: str, LocalL: int)
 |  
 |  apply